<a href="https://colab.research.google.com/github/thesalmonification/DSCI400_Revamp/blob/master/BDF_Full_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import mne
import glob
import xml.etree.ElementTree as ET

In [0]:
os.chdir('/media/duncan/DATA/hci_db_raw/Sessions')
subfolders = os.listdir()

In [0]:
#List of EEG channels used in this study
#['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']




for subfolder in subfolders:
    print(subfolder)
    os.chdir(subfolder)
    bdf_files = glob.glob('*.bdf')
    xml_files = glob.glob('session.xml')
    if (len(bdf_files) == 0) or (len(xml_files) == 0):
        os.chdir('..')
        continue #case where no .bdf/.xml file saved so useless to keep
    
    
    #Parse the .xml file and get the emotional attributes
    #Save the file as a .csv
    root = ET.parse('session.xml').getroot()
    root_dict = root.attrib
    emotion_dict = {k:root_dict[k] for k in ('feltEmo','feltArsl','feltVlnc','feltCtrl','feltPred','mediaFile') if k in root_dict}
    df = pd.DataFrame.from_dict(emotion_dict,orient='index',columns=['Value'])
    df.index.name = 'Label'
    
    #df.to_csv('/media/duncan/DATA/hci_db_raw/Label_CSV/'+subfolder+'_labels.csv') #outdated, will no longer use csv
    df.to_hdf('/media/duncan/DATA/hci_db_raw/Label_Data.h5',key='t' + subfolder+'_labels',complevel=4,format='table')
    
    
    
    

    #In the other case, I have a single .bdf/.xml file pair to parse 
    #I want to build .csv files to replace the .bdf files for every trial
    #Open the raw data
    raw = mne.io.read_raw_bdf(bdf_files[0], preload=True)
    chs = raw.info['ch_names'][:32]
    
    #parse over every EEG channel and save to an array
    #Start a data array by grabbing first channel
    
    #Older style (no trimming)
    #data_array = raw[chs[0]][0]
    
    #Added 2/18 to trim portions where EEG is "shut off"
    ####################################
    ch0 = raw[chs[0]][0][0]
    
    ch0[abs(ch0)<3e-8] = 0
    ch0 = np.trim_zeros(ch0)
    
    ch0 = ch0.reshape(-1,1).T
    
    data_array = ch0
    ####################################
    
    
    
    
    for ch in chs[1:]:   
        
        #Uncomment if you want to see before/after for trimming
        #plt.figure()
        #plt.plot(raw[ch][0][0])
        
        
        #Older style (no trimming)
        #data = raw[ch][0] #the 1st index is just idx samples counting up (ie not needed)
        
        #Added 2/18 to trim portions where EEG is "shut off"
        #####################################
        data = raw[ch][0][0]
        
        data[abs(data)<3e-8] = 0
        data = np.trim_zeros(data)
        
        data = data.reshape(-1,1).T
        
        
        #Uncomment if you want to see before/after for trimming
        #plt.figure()
        #plt.plot(data[0])
        
        
        
        ####################################
        data_array = np.concatenate((data_array,data),axis=0)
        

    
    num_samples = max(data_array.shape)

    #Save the file to CSV using Pandas
    df = pd.DataFrame(data_array,index=chs,columns=range(0,num_samples)).T
    
    #df.to_csv('/media/duncan/DATA/hci_db_raw/Waveform_Trimmed_CSV/'+subfolder + '.csv',index=chs,columns=range(0,num_samples)) #Outdated no longer using CSV
    df.to_hdf('/media/duncan/DATA/Waveform_Data.h5',key='t' + subfolder ,complevel=4,format='table')
    print('-----------------------------------')
    os.chdir('..')

10
-----------------------------------
1042
-----------------------------------
1044
-----------------------------------
1046
-----------------------------------
1048
-----------------------------------
1050
-----------------------------------
1052
-----------------------------------
1054
-----------------------------------
1056
-----------------------------------
1058
-----------------------------------
1060
-----------------------------------
1062
-----------------------------------
1064
-----------------------------------
1066
-----------------------------------
1068
-----------------------------------
1172
-----------------------------------
1174
-----------------------------------
1176
-----------------------------------
1178
-----------------------------------
1180
-----------------------------------
1182
-----------------------------------
1184
-----------------------------------
1186
-----------------------------------
1188
-----------------------------------
1190
-------------

-----------------------------------
554
-----------------------------------
556
-----------------------------------
558
-----------------------------------
560
-----------------------------------
6
-----------------------------------
652
-----------------------------------
654
-----------------------------------
656
-----------------------------------
658
-----------------------------------
660
-----------------------------------
662
-----------------------------------
664
-----------------------------------
666
-----------------------------------
668
-----------------------------------
670
-----------------------------------
672
-----------------------------------
674
-----------------------------------
676
-----------------------------------
678
-----------------------------------
680
-----------------------------------
682
-----------------------------------
684
-----------------------------------
686
-----------------------------------
688
-----------------------------------
690
--

-----------------------------------
2602
-----------------------------------
2604
-----------------------------------
2606
-----------------------------------
2608
-----------------------------------
2610
-----------------------------------
2612
-----------------------------------
2614
-----------------------------------
2616
-----------------------------------
2618
-----------------------------------
262
-----------------------------------
2620
-----------------------------------
2622
-----------------------------------
2624
-----------------------------------
2626
-----------------------------------
2628
-----------------------------------
2630
-----------------------------------
2632
-----------------------------------
2634
-----------------------------------
2636
-----------------------------------
2638
-----------------------------------
264
-----------------------------------
2640
-----------------------------------
266
-----------------------------------
268
--------------------